# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

from firebase_admin import firestore, initialize_app, credentials
cred = credentials.ApplicationDefault()
initialize_app(cred)
db = firestore.client()

In [9]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [17]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]
courses[0]['naziv']

'Završni rad - izrada i odbrana'

## Write courses to rdf